In [ ]:
# 1. Instalação de dependências
!pip install requests GitPython huggingface_hub
# %pip install requests GitPython huggingface_hub   # (Descomente esta linha se estiver usando um notebook Jupyter)

import os
import requests
import json
from git import Repo
from huggingface_hub import InferenceClient

# --- CONFIGURAÇÕES ---
GITHUB_TOKEN = "GITHUB_TOKEN"
HF_TOKEN = "HF_TOKEN"
REPO_URL = "https://github.com/unclecode/crawl4ai"
REPO_NAME = "crawl4ai"
MODEL_ID = "deepseek-ai/DeepSeek-V3" # Nota: Use o endpoint disponível para a V3 no HF

# Inicializar cliente Hugging Face
client = InferenceClient(api_key=HF_TOKEN)


In [2]:
# 2. CLONE DO REPOSITÓRIO
if not os.path.exists(REPO_NAME):
    print(f"Clonando {REPO_NAME}...")
    Repo.clone_from(REPO_URL, REPO_NAME)

# 3. INVESTIGAÇÃO DE CI/CD (Arquivos de Configuração)
def get_workflow_files(repo_path):
    workflow_path = os.path.join(repo_path, ".github/workflows")
    configs = {}
    if os.path.exists(workflow_path):
        for file in os.listdir(workflow_path):
            if file.endswith(('.yml', '.yaml')):
                with open(os.path.join(workflow_path, file), 'r') as f:
                    configs[file] = f.read()[:1000] # Limite para não estourar contexto
    return configs

Clonando crawl4ai...


In [3]:
# 4. ANÁLISE DE PULL REQUESTS (GitHub API)
def get_github_pr_data(repo_owner_repo):
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    url = f"https://api.github.com/repos/{repo_owner_repo}/pulls?state=closed&per_page=10"
    response = requests.get(url, headers=headers)
    prs = response.json()

    pr_summary = []
    for pr in prs:
        pr_summary.append({
            "title": pr['title'],
            "labels": [l['name'] for l in pr['labels']],
            "merged_at": pr['merged_at'],
            "author_association": pr['author_association']
        })
    return pr_summary

# Coleta de dados
workflow_data = get_workflow_files(REPO_NAME)
pr_data = get_github_pr_data("unclecode/crawl4ai")

In [4]:
# 5. GERAR RELATÓRIO COM IA
context = f"""
Arquivos de Workflow encontrados: {list(workflow_data.keys())}
Conteúdo resumido: {json.dumps(workflow_data)}

Histórico de PRs recentes: {json.dumps(pr_data)}
"""

prompt = f"""
Você é um especialista em Engenharia de Software e DevOps.
Analise os dados abaixo do repositório Crawl4AI e elabore um relatório detalhado sobre o ciclo de vida do software (SDLC).

Dados:
{context}

O relatório deve conter:
1. Ferramentas de CI/CD identificadas.
2. Mapeamento do fluxo de automação (testes, deploy, linting).
3. Análise da gestão de mudanças baseada nos Pull Requests.
4. Identificação de riscos ou gargalos (Ex: falta de automação em áreas críticas).
5. Sugestões de melhoria.
"""

print("Gerando relatório com DeepSeek...")
try:
    response = client.chat_completion(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2000
    )
    print("\n" + "="*50)
    print("RELATÓRIO DE CICLO DE VIDA - CRAWL4AI")
    print("="*50)
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Erro ao chamar a API: {e}")

Gerando relatório com DeepSeek...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



RELATÓRIO DE CICLO DE VIDA - CRAWL4AI
# **Relatório de Análise do Ciclo de Vida do Software (SDLC) - Crawl4AI**  

## **1. Ferramentas de CI/CD Identificadas**  
O repositório utiliza **GitHub Actions** como principal ferramenta de CI/CD, com três workflows principais:  

- **`docker-release.yml`**: Automatiza a construção e publicação de imagens Docker.  
- **`release.yml`**: Gerencia o processo de lançamento de versões (tags).  
- **`main.yml`**: Notifica eventos do GitHub (issues, PRs, stars) via Discord e Google Apps Script.  

**Tecnologias e Ferramentas Associadas:**  
- **GitHub Actions** (CI/CD)  
- **Docker** (empacotamento e deploy)  
- **Python 3.12** (ambiente de execução)  
- **Google Apps Script** (integração externa para notificações)  
- **Discord Webhooks** (comunicação de eventos)  

---

## **2. Mapeamento do Fluxo de Automação**  

### **a) Pipeline de Release (`release.yml`)**  
- **Trigger:** Tags no formato `v*` (ex: `v0.8.0`).  
- **Etapas:**  
  1. **Checkout 